In [1]:
import json
import threading
import json
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from urllib.request import urlopen
from PIL import Image
from skimage.io import imread
from skimage.transform import resize
from tqdm import tqdm_notebook as tqdm

from keras.utils.data_utils import Sequence
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint   
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten, Convolution2D, GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

DATA_DIR = "../data/"
NUM_CLASSES = 228
IMAGE_SIZE = 256

C:\Users\matt\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open(DATA_DIR + "test.json") as test:
    test_json = json.load(test)
    
# test_urls = [obj['url'] for obj in test_json['images']]
test_paths = ["../data/test/{}.jpg".format(obj['imageId']) for obj in test_json['images']]
print(test_paths[:3])

['../data/test/1.jpg', '../data/test/2.jpg', '../data/test/3.jpg']


In [3]:
model = Sequential()
model.add(Convolution2D(32, kernel_size=(3, 3),padding='same',input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(Activation('relu'))
model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='sigmoid'))

model.load_weights(DATA_DIR + "model.best.256.hdf5")

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 127, 127, 64)      18496     
__________

In [14]:
class TestBatchSequence(Sequence):
    def __init__(self, x_set, batch_size):
        self.x = x_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        images = np.empty([len(batch_x), IMAGE_SIZE, IMAGE_SIZE, 3])
        for i, path in enumerate(batch_x):
            try:
                image = np.array(Image.open(path))
            except Exception as e:
                print(e)
                output = [1]*(IMAGE_SIZE*IMAGE_SIZE*3)
                output = np.array(output).reshape(IMAGE_SIZE,IMAGE_SIZE,3).astype('uint8')
                image = Image.fromarray(output).convert('RGB')
            images[i, ...] = image
        return images 

In [16]:
%%time
datagen = ImageDataGenerator(rescale = 1. / 255)

BATCH = 64
STEPS = len(test_paths) // BATCH

test_seq = TestBatchSequence(test_paths, BATCH)

# test_gen = datagen.flow_from_directory(
#     "../data/test",
#     target_size = (256, 256),
#     batch_size = BATCH,
#     class_mode = None,
#     shuffle = False
# )

probs = model.predict_generator(
    test_seq,
    steps = STEPS + 1,
    workers = 5,
    verbose = 1
)


621/621 [==============================] - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1:

In [28]:
# images = [np.array(Image.open(path)) for path in test_paths[:1000]]
# subset = test_paths[:1000]
images = np.empty([len(test_paths), IMAGE_SIZE, IMAGE_SIZE, 3], dtype='uint8')
for i, path in enumerate(test_paths):
    try:
        image = np.array(Image.open(path))
    except Exception as e:
        print(e)
        output = [1]*(IMAGE_SIZE*IMAGE_SIZE*3)
        output = np.array(output).reshape(IMAGE_SIZE,IMAGE_SIZE,3).astype('uint8')
        image = Image.fromarray(output).convert('RGB')
    images[i, ...] = image

evals = model.predict_proba(images)

In [29]:
print(probs.shape)
print(evals.shape)

for i in range(3):
    print("Eval: {}".format(evals[i]))
    print("Prob: {}".format(probs[i]))
    print("\n\n")

(39706, 228)
(39706, 228)
Eval: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Prob: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [6]:
def url_to_image(url):
    try:
        resp = urlopen(url)
        image = np.asarray(bytearray(resp.read()), dtype='uint8')
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    except Exception as e: 
        print(e)
        output = [1]*(IMAGE_SIZE*IMAGE_SIZE*3)
        output = np.array(output).reshape(IMAGE_SIZE,IMAGE_SIZE,3).astype('uint8')
        image = Image.fromarray(output).convert('RGB')        
    # image = image[...,::-1]
    image = np.array(image)
    image = resize(image, (IMAGE_SIZE, IMAGE_SIZE))    
    return image

def path_to_image(path):
    image = Image.open(path)
    image = np.array(image)
    image = resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    return image

In [7]:
with open("submission.csv","w") as f:
    f.write("image_id, label_id\n")
    for i, image_url in tqdm(enumerate(test_urls), total = len(test_urls)):
        # image = url_to_image(image_url)
        image = path_to_image(DATA_DIR + "test/{}.jpg".format(i + 1))
        prob = model.predict_proba(image.reshape(1, IMAGE_SIZE, IMAGE_SIZE, 3))
        sorted_args = np.argsort(prob)[0][::-1][:10]
        output_labels = " ".join(str(x) for x in sorted_args)
        # print("{}, {}\n".format(i + 1, output_labels))
        f.write("{}, {}\n".format(i + 1, output_labels))

HBox(children=(IntProgress(value=0, max=39706), HTML(value='')))

C:\Users\matt\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
